In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from datetime import datetime, date, time
import warnings
warnings.filterwarnings('ignore')

# 1. LOADING THE PLOTS INFORMATION

## 1.2 STRUCTURAL PROPERTIES

In [2]:
def maxim(T,total_time,variable):
    t = 0
    maximum = 0
    delta_t = total_time/T
    for j in range (T):
        maximum_new = variable[j]
        if maximum_new > maximum:
            maximum = maximum_new
        
        t = t + delta_t
        
    return maximum

In [3]:
def minim(T,total_time,variable):
    t = 0
    minimum = 0
    delta_t = total_time/T
    for j in range (T):
        minimum_new = variable[j]
        if minimum_new < minimum:
            minimum = minimum_new
        
        t = t + delta_t
        
    return minimum

In [4]:
def amplitud(maximum,minimum):
    A = maximum-minimum
    return A

# 2. DEFINING VARIABLES AND ARRAYS TO STORE DATA

In [5]:
#Creating the 6 arrays neededs to store the infromation in each time step.

T = 100000 #Number of points that time is discretized

N = 100 #Number of points that the airfoil is discretized

#Dynamic velocity arrays:
V_0_new = np.zeros(N+2)
V_1_new = np.zeros(N+2)
V_0_old = np.zeros(N+2)
V_1_old = np.zeros(N+2)

#Moments arrays:
M_0_new = np.zeros(N+2)
M_1_new = np.zeros(N+2)
M_0_old = np.zeros(N+2)
M_1_old = np.zeros(N+2)

#Centrifual term arrays:
S_0_new = np.zeros(N+1)
S_1_new = np.zeros(N+1)
S_0_old = np.zeros(N+1)
S_1_old = np.zeros(N+1)

#Dynamic displacement arrays:
q_0_new = np.zeros(N+2)
q_1_new = np.zeros(N+2)
q_0_old = np.zeros(N+2)
q_1_old = np.zeros(N+2)

#Centrifugal force array:
N_force = np.zeros(N+1)

#Radial position in the blade array:
Position = np.zeros(N+2)

#Time position in the simulation array:
Time = np.zeros(T+1)

#Root bending moments array:
Root_M_0 = np.zeros(T+1)
Root_M_1 = np.zeros(T+1)

#Tip velocity of the displacements array:
Tip_V_0 = np.zeros(T+1)
Tip_V_1 = np.zeros(T+1)

#Tip displacements array:
Tip_q_0 = np.zeros(T+1)
Tip_q_1 = np.zeros(T+1)

#Force damped array:
Damped = np.zeros(T+1)

#Force at the tip:
Force_tip = np.zeros(N+2)

In [6]:
M_0_new

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# 3. MAIN PROGRAM

In [ ]:
#Define time and length steps:

total_time = 10.0
t = 0
#A = -100 #Amplitud aerodynamic force
B = 0 #Amplitud aerodynamic force
#First we need to define all the properties for the i position:
       
EI_00 = (10**10)*1.5
EI_11 = (10**10)*1.5
EI_01 = 0
EI_10 = 0
twist = 0
rho = 500.0
F_a_0 = B
#F_a_1 = A
g_0 = 0.0
g_1 = 0.0

beta = 0.5 #Damped constant
A_q = amplitud(maxim(T,total_time,Tip_q_1),minim(T,total_time,Tip_q_1)) #AMPLITUD dynamic dispalcement response in the tip
A_m = amplitud(maxim(T,total_time,Root_M_1),minim(T,total_time,Root_M_1)) #AMPLITUD dynamic moment response in the root

#Program loop iterations

for j in range(T):
    
    delta_t = total_time/T
    r = 0
    F_a_1 = -100.0#*(math.exp(-t)) #Amplitud aerodynamic force
    #Damped[j] = F_a_1 
    #F_a_1 = A*math.sin(2*3.14*t)
    #Force_tip[N+1] = F_a_1
    
    for i in range(1,N+1):
        
        
        R = 62.5
        delta_r = R/(N-1)
       
        
        
        #Secondly calculate new values of data from the old step time:
        
        
        V_0_new[i] = V_0_old[i] + delta_t*((-1/rho)*((M_0_old[i+1]-(2*M_0_old[i])+M_0_old[i-1])/delta_r**2)+(F_a_0/rho)+ g_0)
        V_1_new[i] = V_1_old[i] + delta_t*((-1/rho)*((M_1_old[i+1]-(2*M_1_old[i])+M_1_old[i-1])/delta_r**2)+(F_a_1/rho)+ g_1)
        
        #Boundary conditions at the ROOT position: 
        V_0_new[0] = 0
        V_1_new[0] = 0
        V_0_new[1] = 0
        V_1_new[1] = 0
        
        M_0_new[i] = M_0_old[i] + delta_t*((EI_00*((V_0_new[i+1]-(2*V_0_new[i])+V_0_new[i-1])/delta_r**2)))#+EI_01*((V_1_new[i+1]-(2*V_1_new[i])+V_1_new[i-1])/delta_r**2)))
        M_1_new[i] = M_1_old[i] + delta_t*((EI_11*((V_1_new[i+1]-(2*V_1_new[i])+V_1_new[i-1])/delta_r**2)))#+EI_10*((V_0_new[i+1]-(2*V_0_new[i])+V_0_new[i-1])/delta_r**2)))
    
        
        
        #Boundary conditions at the TIPS position:
        M_0_new[N+1] = 0
        M_1_new[N+1] = 0
        M_0_new[N] = 0
        M_1_new[N] = 0
        
        
        q_0_new[i] = q_0_old[i] + delta_t*((V_0_old[i]))
        q_1_new[i] = q_1_old[i] + delta_t*((V_1_old[i]))
        
        #Position control:
        
        Position[i] = r
        r = r + delta_r
    
    #Upload data to new time-step:
    V_0_old = V_0_new 
    V_1_old = V_1_new 
    M_0_old = M_0_new 
    M_1_old = M_1_new 
    q_0_old = q_0_new
    q_1_old = q_1_new
    
    #Store data to plot: 
    
    
    Root_M_0[j] = M_0_new[1]
    Root_M_1[j] = M_1_new[1]
    #Root_M_1[j] = (M_1_new[1]+(A_m/2))*math.exp(-beta*t)-(A_m/2)
    Tip_V_0[j] = V_0_new[N]
    Tip_V_1[j] = V_0_new[N]
    Tip_q_0[j] = q_0_new[N]
    Tip_q_1[j] = q_1_new[N]
    #Tip_q_1[j] = ((q_1_new[N]+(A_q/2))*math.exp(-beta*t)-(A_q/2))
    
    #Time control:
    
    Time[j] = t
    t = t + delta_t
    


In [ ]:
maxim(T,total_time,Tip_q_1)

In [ ]:
minim(T,total_time,Tip_q_1)

In [ ]:
amplitud(maxim(T,total_time,Tip_q_1),minim(T,total_time,Tip_q_1))

In [ ]:
maxim(T,total_time,Root_M_1)

In [ ]:
minim(T,total_time,Root_M_1)

In [ ]:
amplitud(maxim(T,total_time,Root_M_1),minim(T,total_time,Root_M_1))

In [ ]:
Tip_q_1

In [ ]:
Position

In [ ]:
Tip_q_1[j]

In [ ]:
M_1_new

In [ ]:
Root_M_1

# 4. RESULTS & PLOTS

In this part, we will make different plots to be able to see in a better way the results obtained with the program developed.

First we will need to create the new dataframes to be able to

In [ ]:
#Time step minimum calculation to obtain a consistent result that should ensure stability:


In [ ]:
#Damped force plot to be sure of the final behaviour:

#Creating the plot figure:

x = Time[:-1]
y = Damped[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Damped force applied [N/m]')
plt.xlabel('Time [s]')
plt.title('Damped force applied');

## 4.1.1 TIP DISPLACEMENT OVER THE TIME

In [ ]:
#Creating the plot figure:

x = Time[:-1]
y = Tip_q_0[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Tip Displacement direction 0 [m]')
plt.xlabel('Time [s]')
plt.title('Tip Displacement direction 0 over the time');

In [ ]:
#Creating the plot figure:

x = Time[:-1]
y = Tip_q_1 [:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Tip Displacement direction 1 [m]')
plt.xlabel('Time [s]')
plt.title('Tip Displacement direction 1 over the time');

In [ ]:
np.average(Tip_q_1)

## 4.1.2 DISPLACEMENT IN THE BLADE

In [ ]:
#Creating the plot figure:

x = Position[:-1]
y = q_0_new[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Displacement direction 0 [m]')
plt.xlabel('Blade radius [m]')
plt.title('Displacement along spanwise on direction 0');

In [ ]:
#Creating the plot figure:

x = Position[:-1]
y = q_1_new[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Displacement direction 1 [m]')
plt.xlabel('Blade radius [m]')
plt.title('Displacement along spanwise on direction 1');

## 4.2.1 ROOT BENDING MOMENT OVER THE TIME

In [ ]:
#Creating the plot figure:

x = Time[:-1]
y = Root_M_0[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Root bending Moment 0 [N·m]')
plt.xlabel('Time [s]')
plt.title('Root bending Moment 0');

In [ ]:
#Creating the plot figure:

x = Time[:-1]
y = Root_M_1[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Root bending Moment 1 [N·m]')
plt.xlabel('Time [s]')
plt.title('Root bending Moment 1');

In [ ]:
np.average(Root_M_1)

## 4.2.2 BENDING MOMENT IN THE BLADE

In [ ]:
#Creating the plot figure:

x = Position[:-1]
y = M_0_new[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Bending Moment 0 [N·m]')
plt.xlabel('Blade radius [m]')
plt.title('Bending Moment 0 over the spanwise');

In [ ]:
#Creating the plot figure:

x = Position[:-1]
y = M_1_new[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Bending Moment 1 [N·m]')
plt.xlabel('Blade radius [m]')
plt.title('Bending Moment 1 over the spanwise');

In [ ]:
M_1_new

In [ ]:
Root_M_1

## 4.3.1 TIP VELOCITY DISPLACEMENT OVER THE TIME 

In [ ]:
#Creating the plot figure:

x = Time[:-1]
y = Tip_V_0[:-1]

plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Tip Velocity direction 0 [m/s]')
plt.xlabel('Time [s]')
plt.title('Tip Velocity direction 0');

In [ ]:
#Creating the plot figure:

x = Time[:-1]
y = Tip_V_1[:-1]

plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Tip Velocity direction 1 [m/s]')
plt.xlabel('Time [s]')
plt.title('Tip Velocity direction 1');

## 4.3.2 VELOCITY OF DISPLACEMENT IN THE BLADE

In [ ]:
#Creating the plot figure:

x = Position[:-1]
y = V_0_new[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Tip Velocity direction 0 [m/s]')
plt.xlabel('Blade radius [m]')
plt.title('Tip Velocity along spanwise on direction 0');

In [ ]:
#Creating the plot figure:

x = Position[:-1]
y = V_1_new[:-1]


plt.plot(x, y, color='blue', linewidth=2)
plt.ylabel('Tip Velocity direction 1 [m/s]')
plt.xlabel('Blade radius [m]')
plt.title('Tip Velocity along spanwise on direction 1');